In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
!pwd

/kaggle/working


## Titanic - Machine Learning from Disaster
### Start here! Predict survival on the Titanic and get familiar with ML basics

Dataset Description
-------------------

### Overview

The data has been split into two groups:

-   training set (train.csv)
-   test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the "ground truth") for each passenger. Your model will be based on "features" like passengers' gender and class. You can also use [feature engineering ](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

### Data Dictionary

| Variable | Definition | Key |
|---|---|---|
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex |  |
| Age | Age in years |  |
| sibsp | # of siblings / spouses aboard the Titanic |  |
| parch | # of parents / children aboard the Titanic |  |
| ticket | Ticket number |  |
| fare | Passenger fare |  |
| cabin | Cabin number |  |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes

pclass: A proxy for socio-economic status (SES)\
1st = Upper\
2nd = Middle\
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...\
Sibling = brother, sister, stepbrother, stepsister\
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...\
Parent = mother, father\
Child = daughter, son, stepdaughter, stepson\
Some children travelled only with a nanny, therefore parch=0 for them.

### SOLUTION
---

In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data  = pd.read_csv("/kaggle/input/titanic/test.csv")
test_ids = test_data["PassengerId"]

In [4]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  10692 	SHAPE :  (891, 12)
Test  -> SIZE :  4598 	SHAPE :  (418, 11)


In [5]:
train_data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
52,53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
625,626,0,1,"Sutton, Mr. Frederick",male,61.0,0,0,36963,32.3208,D50,S
783,784,0,3,"Johnston, Mr. Andrew G",male,NaN,1,2,W./C. 6607,23.4500,NaN,S
419,420,0,3,"Van Impe, Miss. Catharina",female,10.0,0,2,345773,24.1500,NaN,S
569,570,1,3,"Jonsson, Mr. Carl",male,32.0,0,0,350417,7.8542,NaN,S
499,500,0,3,"Svensson, Mr. Olof",male,24.0,0,0,350035,7.7958,NaN,S
857,858,1,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S
471,472,0,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,NaN,S


In [6]:
test_data.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
349,1241,2,"Walcroft, Miss. Nellie",female,31.0,0,0,F.C.C. 13528,21.0000,NaN,S
170,1062,3,"Lithman, Mr. Simon",male,NaN,0,0,S.O./P.P. 251,7.5500,NaN,S
196,1088,1,"Spedden, Master. Robert Douglas",male,6.0,0,2,16966,134.5000,E34,C
242,1134,1,"Spedden, Mr. Frederic Oakley",male,45.0,1,1,16966,134.5000,E34,C
223,1115,3,"Karlsson, Mr. Einar Gervasius",male,21.0,0,0,350053,7.7958,NaN,S
163,1055,3,"Pearce, Mr. Ernest",male,NaN,0,0,343271,7.0000,NaN,S
377,1269,2,"Cotterill, Mr. Henry Harry""""",male,21.0,0,0,29107,11.5000,NaN,S
136,1028,3,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C
27,919,3,"Daher, Mr. Shedid",male,22.5,0,0,2698,7.2250,NaN,C
34,926,1,"Mock, Mr. Philipp Edmund",male,30.0,1,0,13236,57.7500,C78,C


In [7]:
# Creating Function to Drop Columns from the DataFrame
# and Fill missing rows where data in not present

def dataset_cleaner(data):
    # Dropping Following columns as they will be unnecessary for predictions
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

    # Filling missing Values
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    
    # U --> Unknown Token
    data.Embarked.fillna("U", inplace=True)

    return data


train_data = dataset_cleaner(train_data)
test_data  = dataset_cleaner(test_data)

In [8]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  7128 	SHAPE :  (891, 8)
Test  -> SIZE :  2926 	SHAPE :  (418, 7)


In [9]:
train_data.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
285,0,3,male,33.0,0,0,8.6625,C
526,1,2,female,50.0,0,0,10.5000,S
537,1,1,female,30.0,0,0,106.4250,C
334,1,1,female,28.0,1,0,133.6500,S
30,0,1,male,40.0,0,0,27.7208,C
372,0,3,male,19.0,0,0,8.0500,S
387,1,2,female,36.0,0,0,13.0000,S
483,1,3,female,63.0,0,0,9.5875,S
166,1,1,female,28.0,0,1,55.0000,S
278,0,3,male,7.0,4,1,29.1250,Q


In [10]:
test_data.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
354,3,female,0.17,1,2,20.5750,S
342,3,male,27.00,1,9,69.5500,S
29,3,male,27.00,2,0,21.6792,C
395,1,female,18.00,1,0,60.0000,S
303,3,male,24.00,0,0,8.6625,S
155,3,male,24.00,0,0,7.5500,S
244,3,male,27.00,1,2,23.4500,S
78,2,male,30.00,0,0,13.0000,S
103,3,male,26.00,0,0,7.7750,S
174,3,male,40.00,1,5,31.3875,S


In [11]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()

def encoder(data):
    # Encoding the following Columns values to encoded integers
    cols = ["Sex", "Embarked"]

    for col in cols:
        data[col] = lab_enc.fit_transform(data[col])
    

encoder(train_data)
encoder(test_data)        

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x = train_data.drop("Survived", axis=1)
y = train_data["Survived"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
classifier = LogisticRegression(random_state=3, max_iter=3000)
classifier = classifier.fit(x_train, y_train)

pred = classifier.predict(x_test)

In [14]:
from sklearn.metrics import accuracy_score

print("Classifier Accuracy :",accuracy_score(y_test, pred))

Classifier Accuracy : 0.8100558659217877


In [15]:
submit_pred = classifier.predict(test_data)

submit_df = pd.DataFrame({"PassengerId":test_ids.values,
                        "Survived": submit_pred})

submit_df.to_csv("submission.csv", index=False)
